In [1]:
import random
import numpy as np
import tensorflow as tf

random.seed(0)
np.random.seed(0)
tf.random.set_seed(0)

In [2]:
from keras import Sequential
from keras.layers import *
from keras.losses import BinaryCrossentropy
from keras.callbacks import EarlyStopping
from sklearn.naive_bayes import GaussianNB

from src.experiment import *
from src.features.encodings import ps
from src.data import load_psi, load_2ome, Species

In [4]:
human_test = load_psi(Species.human, independent=True)
human_train = load_psi(Species.human, independent=False)

yeast_test = load_psi(Species.yeast, independent=True)
yeast_train = load_psi(Species.yeast, independent=False)

In [5]:
encoder = ps.Encoder(k=3)

In [6]:
human_x_train = encoder.fit_transform(human_train.samples)
human_y_train = human_train.targets
human_x_test = encoder.fit_transform(human_test.samples)
human_y_test = human_test.targets

In [7]:
yeast_x_train = encoder.fit_transform(yeast_train.samples)
yeast_y_train = yeast_train.targets
yeast_x_test = encoder.fit_transform(yeast_test.samples)
yeast_y_test = yeast_test.targets

In [14]:
human_model = Sequential([
    Dense(1216, input_dim=1216, activation='relu'),
    Dropout(0.8),
    Dense(500, activation='relu'),
    Dropout(0.8),
    Dense(1, activation='sigmoid'),
])

human_model.compile(
    loss=BinaryCrossentropy(from_logits=False),
    optimizer='adam',
    metrics=['accuracy'],
)

early_stopping = EarlyStopping(
    patience=5,
    mode='min',
    restore_best_weights=True
)

human_model.fit(human_x_train, human_y_train,
                epochs=20, callbacks=[early_stopping],
                validation_data=(human_x_test, human_y_test))

Epoch 1/10
31/31 [==============================] - 0s 6ms/step - loss: 0.7231 - accuracy: 0.5182 - val_loss: 0.6738 - val_accuracy: 0.5850
Epoch 2/10
31/31 [==============================] - 0s 5ms/step - loss: 0.6813 - accuracy: 0.5808 - val_loss: 0.6453 - val_accuracy: 0.6350
Epoch 3/10
31/31 [==============================] - 0s 4ms/step - loss: 0.6000 - accuracy: 0.6717 - val_loss: 0.6170 - val_accuracy: 0.6750
Epoch 4/10
31/31 [==============================] - 0s 4ms/step - loss: 0.5067 - accuracy: 0.7646 - val_loss: 0.6084 - val_accuracy: 0.7100
Epoch 5/10
31/31 [==============================] - 0s 5ms/step - loss: 0.4206 - accuracy: 0.8030 - val_loss: 0.6493 - val_accuracy: 0.6700
Epoch 6/10
31/31 [==============================] - 0s 5ms/step - loss: 0.3330 - accuracy: 0.8657 - val_loss: 0.6959 - val_accuracy: 0.6500
Epoch 7/10
31/31 [==============================] - 0s 5ms/step - loss: 0.2446 - accuracy: 0.9030 - val_loss: 0.7642 - val_accuracy: 0.6400
Epoch 8/10
31/31 [==

In [16]:
yeast_model = Sequential([
    Dense(1856, input_dim=1856, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.8),
    Dense(1, activation='sigmoid'),
])

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    mode='min',
    restore_best_weights=True,
)

yeast_model.compile(loss=BinaryCrossentropy(from_logits=False), optimizer='adam', metrics=['accuracy'])

yeast_model.fit(yeast_x_train, yeast_y_train,
                epochs=40, callbacks=[early_stopping],
                validation_data=(yeast_x_test, yeast_y_test))

Epoch 1/40
20/20 [==============================] - 0s 9ms/step - loss: 0.7018 - accuracy: 0.5239 - val_loss: 0.6743 - val_accuracy: 0.5600
Epoch 2/40
20/20 [==============================] - 0s 6ms/step - loss: 0.5991 - accuracy: 0.6338 - val_loss: 0.6416 - val_accuracy: 0.6300
Epoch 3/40
20/20 [==============================] - 0s 6ms/step - loss: 0.5386 - accuracy: 0.7038 - val_loss: 0.6103 - val_accuracy: 0.6550
Epoch 4/40
20/20 [==============================] - 0s 5ms/step - loss: 0.4280 - accuracy: 0.7707 - val_loss: 0.6213 - val_accuracy: 0.6200
Epoch 5/40
20/20 [==============================] - 0s 6ms/step - loss: 0.2923 - accuracy: 0.8694 - val_loss: 0.7054 - val_accuracy: 0.6400
Epoch 6/40
20/20 [==============================] - 0s 6ms/step - loss: 0.1820 - accuracy: 0.9331 - val_loss: 0.7823 - val_accuracy: 0.6450
Epoch 7/40
20/20 [==============================] - 0s 6ms/step - loss: 0.1260 - accuracy: 0.9618 - val_loss: 0.9241 - val_accuracy: 0.6350
Epoch 8/40
20/20 [==

In [23]:
def select_features(features):
    selection = [4, 21, 20, 0, 5, 65, 64, 1, 17, 16, 60, 15, 25, 54, 3, 11, 46,
                 41, 40, 39, 7, 2, 42, 36, 31, 12, 24, 35, 18, 62, 61, 53, 8, 50,
                 56, 43, 19, 28, 9, 59, 37, 38, 33, 48, 44, 45, 63, 10, 22, 27,
                 49, 57, 55, 14, 51, 47, 52, 26, 23, 29, 30, 13, 6, 58]
    imp_features = list(map(lambda x: f'ps_3_{x + 1}', selection))
    return features[imp_features]

In [24]:
human_classifier = GaussianNB()

human_classifier.fit(select_features(human_x_train), human_y_train)
human_classifier.score(select_features(human_x_test), human_y_test)
None

In [25]:
yeast_classifier = GaussianNB()

yeast_classifier.fit(select_features(yeast_x_train), yeast_y_train)
yeast_classifier.score(select_features(yeast_x_test), yeast_y_test)
None

In [30]:
generate_latex_report(
    Report.create_report(human_model, (human_x_test, human_y_test), True),
    'human_nn_test',
    Path('ps3_report'),
    True
)

generate_latex_report(
    Report.create_report(yeast_model, (yeast_x_test, yeast_y_test), True),
    'yeast_nn_test',
    Path('ps3_report'),
    True
)

generate_latex_report(
    Report.create_report(human_classifier, (select_features(human_x_test), human_y_test)),
    'human_gnb_test',
    Path('ps3_report'),
    True
)

generate_latex_report(
    Report.create_report(yeast_classifier, (select_features(yeast_x_test), yeast_y_test)),
    'yeast_gnb_test',
    Path('ps3_report'),
    True
)

7/7 [==============================] - 0s 2ms/step
note: Running TeX ...
note: Rerunning TeX because "human_nn_test.aux" changed ...
note: Running xdvipdfmx ...
note: Writing `ps3_report/human_nn_test.pdf` (27.08 KiB)
note: Skipped writing 1 intermediate files (use --keep-intermediates to keep them)
7/7 [==============================] - 0s 2ms/step
note: Running TeX ...
note: Rerunning TeX because "yeast_nn_test.aux" changed ...
note: Running xdvipdfmx ...
note: Writing `ps3_report/yeast_nn_test.pdf` (27.30 KiB)
note: Skipped writing 1 intermediate files (use --keep-intermediates to keep them)
note: Running TeX ...
note: Rerunning TeX because "human_gnb_test.aux" changed ...
note: Running xdvipdfmx ...
note: Writing `ps3_report/human_gnb_test.pdf` (26.10 KiB)
note: Skipped writing 1 intermediate files (use --keep-intermediates to keep them)
note: Running TeX ...
note: Rerunning TeX because "yeast_gnb_test.aux" changed ...
note: Running xdvipdfmx ...
note: Writing `ps3_report/yeast_gnb